In [1]:
import pickle 
import pandas as pd 
import matplotlib.pyplot as plt 
import itertools
import numpy as np
from scipy.stats import pearsonr

# クラスタリング
from sklearn.cluster import KMeans

with open('../../data/Hazumi_features/Hazumi1911_features_small.pkl', mode='rb') as f:
    SS, TS, SP, TP, text, audio, visual, vid = pickle.load(f, encoding='utf-8')

def calc_p(df):
    for i, j in itertools.combinations(df, 2): #組み合わせでforループを実行
        x = df.loc[:,[i]].values
        y = df.loc[:,[j]].values
        a, b = pearsonr(np.ravel(x), np.ravel(y)) # リストを整形し相関係数:aとp値:bの計算
        if 0 != b.round(10) < 0.05: # p値 < 0.05
            print("=" * 45)
            print(i + "----" + j)
            print("相関係数:", a.round(4))
            print("p値:", b.round(4))
            plt.show()

In [2]:
# データの偏り（Highクラスの割合と性格特性の相関係数
High_rate = {}
E, A, C, N, O = {}, {}, {}, {}, {}
for id in vid:
    count = [0, 0]
    for ss in SS[id]:
        count[ss] += 1
    High_rate[id] = round(count[1] / sum(count), 3)
    P = SP
    E[id], A[id], C[id], N[id], O[id] = P[id][0], P[id][1], P[id][2], P[id][3], P[id][4]
print(High_rate)
df = pd.DataFrame(
    {
        "High Rate": High_rate,
        "E": E,
        "A": A,
        "C": C,
        "N": N,
        "O": O
    }
)

calc_p(df)
df.corr()

{'1911F2001': 0.479, '1911F3001': 0.27, '1911F4001': 0.082, '1911F5001': 0.891, '1911F6001': 0.099, '1911F7002': 0.897, '1911M2001': 0.792, '1911M4001': 0.126, '1911M5001': 0.644, '1911M6001': 0.585, '1911M7001': 0.182}
A----O
相関係数: -0.7884
p値: 0.0039


,High Rate,E,A,C,N,O
High Rate,1.000000,-0.117865,-0.081695,-0.019765,0.094790,0.215252
E,-0.117865,1.000000,0.093967,0.386207,-0.077735,0.259646
A,-0.081695,0.093967,1.000000,0.389727,0.043711,-0.788405
C,-0.019765,0.386207,0.389727,1.000000,-0.532491,-0.234974
N,0.094790,-0.077735,0.043711,-0.532491,1.000000,-0.157297
O,0.215252,0.259646,-0.788405,-0.234974,-0.157297,1.000000


In [3]:
# 性格特性によるクラスタリング
n_clusters = 4
traits = ["E", "A", "C", "N", "O"]
df = pd.DataFrame.from_dict(TP, orient='index', columns=traits)
# sc = StandardScaler()
# df_1911 = sc.fit_transform(df_1911)
# df_1911 = pd.DataFrame(df_1911, columns=columns)

model = KMeans(n_clusters=n_clusters, random_state=0) 
model.fit(df)
pred = model.predict(df)


df['cluster'] = pred
df
# # show_boxplotv2(df_1911)
# results = [[] for _ in range(9)]
# for index, row in df.iterrows():
#     results[int(row["cluster"])].append(row["E": "O"].values.tolsit())
# print(results)
# cluster = {} 
# for i, id in enumerate(vid1911):
#     cluster[id] = df_1911.loc[i, "cluster"]
# with open('../../data/Hazumi_features/Hazumi1911_features_cluster.pkl', mode='wb') as f:
#     pickle.dump((SS1911, TS1911, SP1911, TP1911, Text1911, Audio1911, Visual1911, cluster, vid1911), f)

/home/ryoyanagimoto/anaconda3/envs/personality/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,E,A,C,N,O,cluster
1911F2001,12.4,12.4,6.6,8.4,9.8,2
1911F3001,7.2,13.2,11.0,5.2,8.6,3
1911F4001,11.0,12.6,10.4,4.6,9.8,0
1911F5001,11.0,13.4,11.0,4.2,10.2,0
1911F6001,8.6,11.6,11.4,5.0,9.0,3
1911F7002,10.2,10.8,8.4,7.6,6.8,2
1911M2001,6.0,9.4,8.6,6.8,6.8,1
1911M4001,8.4,13.0,10.0,5.2,8.2,3
1911M5001,9.0,12.6,9.8,4.0,8.6,3
1911M6001,10.2,11.0,9.2,4.4,9.8,0


In [15]:
# 性格特性によるクラスタリング (大規模)

with open('../../data/Hazumi_features/Hazumi1911_features.pkl', mode='rb') as f:
    _, _, _, TP_1911, _, _, _, _ = pickle.load(f, encoding='utf-8')
with open('../../data/Hazumi_features/Hazumi2010_features.pkl', mode='rb') as f:
    _, _, _, TP_2010, _, _, _, _ = pickle.load(f, encoding='utf-8')

TP = {} 
TP.update(TP_1911)
TP.update(TP_2010)

n_clusters = 9
traits = ["E", "A", "C", "N", "O"]

df = pd.DataFrame.from_dict(TP, orient='index', columns=traits)
# sc = StandardScaler()
# df_1911 = sc.fit_transform(df_1911)
# df_1911 = pd.DataFrame(df_1911, columns=columns)

model = KMeans(n_clusters=n_clusters, random_state=0) 
model.fit(df)
pred = model.predict(df)


df['cluster'] = pred
df.loc[vid, 'cluster']

/home/ryoyanagimoto/anaconda3/envs/personality/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


1911F2001    0
1911F3001    1
1911F4001    6
1911F5001    6
1911F6001    1
1911F7002    7
1911M2001    3
1911M4001    1
1911M5001    1
1911M6001    2
1911M7001    3
Name: cluster, dtype: int32